In [32]:
from skimage import io
import numpy as np
import matplotlib.image as mimg
import cv2
from skimage.color import rgb2gray

In [33]:
def discreteContrast(filteredImage):
    shape = np.shape(filteredImage)
    img = filteredImage.copy()
    valueList = np.unique(img)
    valueDict = dict()
    for i in range(0, len(valueList)):
        valueDict.update({valueList[i] : i})
    for v in range(shape[0]):
        for u in range(shape[1]):
            img[v][u] = valueDict.get(filteredImage[v][u])
    return img

In [34]:
def invertedBinaryImage(image):
    img = image.copy()
    shape = np.shape(img)
    if len(shape) > 2:
        if shape[2] > 3:
            img = rgb2gray(img[:,:,0:3])*255
        else:
            img = rgb2gray(img)*255
    b_img = np.zeros((shape[0], shape[1]))
    for v in range(0, shape[0]):
        for u in range(0, shape[1]):
            if img[v][u] != 255:
                b_img[v][u] = 1
    return b_img

In [35]:
def invertedBinaryImage2(image):
    ret, converted = cv2.threshold(image, 1, 1, cv2.THRESH_BINARY)
    return invert(converted)

In [36]:
def invert(image):
    img = image.copy()
    img = (img * -1) + np.amax(image)
    return img

In [37]:
def labeledNeighbors(image, x, y, n=8):
    nbrs = list()
    if x - 1 >= 0 and image[y][x-1] > 1:
        nbrs.append((x-1, y))
    if n == 4:
        if y - 1 >= 0 and image[y-1][x] > 1:
            nbrs.append((x, y-1))
        return nbrs
    r = -1
    v = 2
    if x - 1 < 0:
        r = 0
    if x + 1 >= np.shape(image)[1]:
        v = 1
    if y - 1 >= 0:
        for i in range(r, v):
            if image[y-1][x+i] > 1:
                nbrs.append((x+i, y-1))
    return nbrs

In [38]:
def zeroNeighbors(image, x, y):
    shape = np.shape(image)
    r = -1
    v = 2
    if y - 1 < 0 and x - 1 < 0:
        return True
    if x - 1 < 0:
        r = 0
    if r == -1 and image[y][x-1] > 0:
        return False
    if x + 1 >= shape[1]:
        v = 1
    if y - 1 < 0:
        return True
    for i in range(r, v):
        if image[y-1][x+i] > 0:
            return False
    return True

In [41]:
def sequentialLabeling(image, n=8):
    img = invertedBinaryImage(image)
    m = 2
    c = list()
    for v in range(0, np.shape(img)[0]):
        for u in range(0, np.shape(img)[1]):
            if img[v][u] == 1:
                nbrs = labeledNeighbors(img, u, v, n)
                if len(nbrs) == 0:
                    img[v][u] = m
                    m = m + 1
                elif len(nbrs) == 1:
                    p = nbrs[0]
                    img[v][u] = img[p[1]][p[0]]
                elif len(nbrs) > 1:
                    p = nbrs.pop(0)
                    k = img[p[1]][p[0]]
                    img[v][u] = k
                    for p in nbrs:
                        ni = img[p[1]][p[0]]
                        if ni != k:
                            c.append({ni, k})
    
    r = list()
    for i in range(2, m):
        r.append({i})
    
    for collisions in c:
        a = collisions.pop()
        b = collisions.pop()
        for labelsets in r:
            if labelsets.issuperset({a}):
                ra = labelsets
            if labelsets.issuperset({b}):
                rb = labelsets
        if ra.isdisjoint(rb):
            ra.update(rb)
            r.remove(rb)
    
    for v in range(0, np.shape(img)[0]):
        for u in range(0, np.shape(img)[1]):
            if img[v][u] > 1:
                for lsets in r:
                    if lsets.issuperset({img[v][u]}):
                        img[v][u] = min(lsets)
    return invert(img)
                    

In [42]:
reg1 = io.imread('regionen1.png')
seqL = sequentialLabeling(reg1)
mimg.imsave('reg1.png', seqL, cmap='gray')
seqLn4 = sequentialLabeling(reg1, n=4)
mimg.imsave('reg1n4.png', seqLn4, cmap='gray')
seqLn4maxContrast = discreteContrast(seqLn4)
mimg.imsave('reg1n4maxContrast.png', seqLn4maxContrast, cmap='gray')

In [30]:
reg2 = io.imread('regionen2.png')
seqL2 = sequentialLabeling(reg2)
mimg.imsave('reg2.png', seqL2, cmap='gray')
seqL2n4 = sequentialLabeling(reg2, n=4)
mimg.imsave('reg2n4.png', seqL2n4, cmap='gray')

In [90]:
quad = io.imread('quadranten.png')
mimg.imsave('quadn8.png', sequentialLabeling(quad), cmap='gray')
mimg.imsave('quadn8bc.png', discreteContrast(sequentialLabeling(quad)), cmap='gray')

### Original vs seq. lab. N8 vs seq. lab N4 vs seq. lab. N4 + better contrast
![orig](regionen1.png)
![sl](reg1.png)
![sln4](reg1n4.png)
![sln4maxCont](reg1n4maxContrast.png)
### Original vs seq. lab. N8 vs seq. lab. N4
![orig](regionen2.png)
![sl](reg2.png)
![sln4](reg2n4.png)  

![](quadranten.png)
![](quadN8.png)
![](quadN8bc.png)